In [1]:
import numpy as np

import os, pickle, time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 이런식의 코드를 막 그냥 작성해버리면 나중에 알아보기 어렵습니다.

- <span style = 'font-size:1.2em;line-height:1.5em'>게다가, Jupyter Notebook형식의 .ipynb로 작성하면 나중에 이 코드를 다시 써먹기가 어렵습니다.</span>

- <span style = 'font-size:1.2em;line-height:1.5em'>좀 헷갈려도 코드를 모듈화하고 스크립트 파일인 .py파일로 변환해줄 필요가 있습니다.</span>

# 필요한 코드들을 모듈화 해보겠습니다.

## 1. Model Class
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 1.</b> (데이터 수 * 784)크기의 행렬(X)을 네트워크에 입력</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 2.</b> 1st hidden layer를 통과.  h1 = Matmul(X, W1) + b1</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>W1: (784*50) 크기의 행렬, b1: (50, )크기의 행렬, h1: (데이터 수 * 50) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 3.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 4.</b> 2nd hidden layer를 통과. h2 = Matmul(h1, W2) + b2</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W2: (50*100) 크기의 행렬, b2 = (100, )크기의 행렬, h2: (데이터 수 * 100) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 5.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 6.</b> output layer를 통과. h3 = Matmul(h2, W3) + b3</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W3: (100*10) 크기의 행렬, b3 = (10, )크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 7.</b> Activation함수 통과 (softmax)</span>


In [2]:
class MyNet(nn.Module):
    def __init__(self, dim_in=784, dim_h1=50, dim_h2=100, dim_out=10):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(dim_in, dim_h1, bias=True)
        self.fc2 = nn.Linear(dim_h1, dim_h2, bias=True)
        self.fc3 = nn.Linear(dim_h2, dim_out, bias=True)
        
    def forward(self, x):
        h1 = self.fc1(x) # Step 1, 2
        h1 = torch.sigmoid(h1) # Step 3
        h2 = self.fc2(h1) # Step 4
        h2 = torch.sigmoid(h2) # Step 5
        out = self.fc3(h2) # Step 6
        out = F.log_softmax(out) # Step 7
        return out

## 2. train() 함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`train()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 지난 batch로부터 계산했던 gradient를 초기화(`zero_grad()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Backpropagation으로 각 parameter의 gradient를 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> Gradient Descent로 parameter값 update</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> `trn_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 8.</b> 데이터 한 개당 평균 train loss 산출</span>

In [3]:
def train(model, data_loader, optimizer, device):
    model.train() # 모델을 학습모드로!
    trn_loss = 0
    for i, (x, y) in enumerate(data_loader):
        # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
        x = x.view(-1, 784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
        y = y.to(device)
        
        # Step 2. gradient 초기화
        optimizer.zero_grad()
        
        # Step 3. Forward Propagation
        y_pred_prob = model(x)
        
        # Step 4. Loss Calculation
        loss = F.nll_loss(y_pred_prob, y, reduction='sum')
        
        # Step 5. Gradient Calculation (Backpropagation)
        loss.backward()
        
        # Step 6. Update Parameter (by Gradient Descent)
        optimizer.step()
        
        # Step 7. trn_loss 변수에 mini-batch loss를 누적해서 합산
        trn_loss += loss.item()
        
    # Step 8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

## 3. evaluate()함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`evaluate()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> 모델이 예측하는 레이블을 산출 (with `torch.argmax()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Minibatch의 실제 레이블(`y`)과 예측 레이블(`y_pred_label`)을 누적하여 저장</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> `eval_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> 데이터 한 개당 평균 evaluation loss와 accuracy 산출</span>

In [4]:
def evaluate(model, data_loader, optimizer, device):
    model.eval() # 모델을 평가모드로!
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.view(-1,784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = F.nll_loss(y_pred_prob, y, reduction='sum')
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

## 4. 매 Epoch에 드는 시간 측정

In [5]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 5. 학습하기

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [6]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
trn_dset = datasets.MNIST(root=data_path, train=True, transform=transform, download=True)
tst_dset = datasets.MNIST(root=data_path, train=False, transform=transform, download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [8]:
model = MyNet(dim_in=784, dim_h1=50, dim_h2=100, dim_out=10)
model = model.to(device)

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [9]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [10]:
N_EPOCHS = 10
LR = 2e-4
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [11]:
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기</span>

In [12]:
my_opt = optim.Adam(model.parameters(), lr = LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [13]:
best_val_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=tst_loader, 
                                  optimizer=my_opt, 
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'{save_dir}/my_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

C:\Users\user\AppData\Local\Temp\ipykernel_3140\773842903.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(out) # Step 7


Epoch: 01 | Time: 0m 12s
	Train Loss: 2.280 | Test Loss: 2.224 | Test Acc: 32.910% 
Epoch: 02 | Time: 0m 12s
	Train Loss: 2.130 | Test Loss: 1.994 | Test Acc: 49.770% 
Epoch: 03 | Time: 0m 12s
	Train Loss: 1.833 | Test Loss: 1.654 | Test Acc: 57.720% 
Epoch: 04 | Time: 0m 12s
	Train Loss: 1.513 | Test Loss: 1.366 | Test Acc: 66.700% 
Epoch: 05 | Time: 0m 12s
	Train Loss: 1.257 | Test Loss: 1.141 | Test Acc: 74.520% 
Epoch: 06 | Time: 0m 12s
	Train Loss: 1.061 | Test Loss: 0.969 | Test Acc: 78.420% 
Epoch: 07 | Time: 0m 12s
	Train Loss: 0.912 | Test Loss: 0.840 | Test Acc: 81.400% 
Epoch: 08 | Time: 0m 12s
	Train Loss: 0.797 | Test Loss: 0.738 | Test Acc: 83.810% 
Epoch: 09 | Time: 0m 12s
	Train Loss: 0.705 | Test Loss: 0.655 | Test Acc: 85.540% 
Epoch: 10 | Time: 0m 12s
	Train Loss: 0.630 | Test Loss: 0.587 | Test Acc: 86.540% 
